In [1]:
ROOT = '/home/anhphantq/otto'
max_session = 64


In [17]:
DATA_FOLDER = ROOT + '/splitted_data'
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')
train_df = pd.read_parquet(DATA_FOLDER + '/train.parquet')


In [2]:
DATA_FOLDER = ROOT + '/data'
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')
train_df = pd.read_parquet(DATA_FOLDER + '/train.parquet')

#remove 1 week from train df LB to retain distribution 
import numpy as np
min_ts = train_df.ts.values.min()
old_index = np.argwhere(train_df.ts.values - min_ts < 3600 * 24 * 7)
old_shape = train_df.shape[0]
old_num_aid = len(np.unique(train_df.aid))
train_df = train_df.drop(old_index.flatten())
num_aid = len(np.unique(train_df.aid))
train_df.shape[0], old_shape,  old_shape - train_df.shape[0], num_aid, old_num_aid, old_num_aid - num_aid

(163555218, 216716096, 53160878, 1839558, 1855603, 16045)

In [3]:
train_df

,session,aid,ts,type
63,0,1629651,1659964140,0
64,0,1768568,1660015113,0
65,0,1318324,1660015128,0
66,0,1840418,1660024261,0
67,0,1813509,1660024355,0
...,...,...,...,...
216716091,12899776,1737908,1661723987,0
216716092,12899777,384045,1661723976,0
216716093,12899777,384045,1661723986,0
216716094,12899778,561560,1661723983,0


In [ ]:
import pandas as pd
test_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/test_v1.parquet')
train_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/train_v1.parquet')


In [ ]:
import pandas as pd
test_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/test_v1.parquet')
train_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/train_v1.parquet')


#Add subsession

In [6]:
import numpy as np

def add_diff(df):
  df = df.sort_values(['session', 'ts'])
  df['diff'] = df['ts'].diff()
  session = df['session'].values
  ends_idx = session[1: ] - session[:-1]
  start_session = np.argwhere(ends_idx > 0) + 1
  start_session  = np.vstack(([0], start_session))
  df.iloc[start_session, -1] = 0
  return df
  

In [7]:
train_df = add_diff(train_df)
train_df

,session,aid,ts,type,diff
63,0,1629651,1659964140,0,0.0
64,0,1768568,1660015113,0,50973.0
65,0,1318324,1660015128,0,15.0
66,0,1840418,1660024261,0,9133.0
67,0,1813509,1660024355,0,94.0
...,...,...,...,...,...
216716091,12899776,1737908,1661723987,0,15.0
216716092,12899777,384045,1661723976,0,0.0
216716093,12899777,384045,1661723986,0,10.0
216716094,12899778,561560,1661723983,0,0.0


In [8]:
test_df = add_diff(test_df)
test_df

,session,aid,ts,type,diff
0,12899779,59625,1661724000,0,0.0
1,12899780,1142000,1661724000,0,0.0
2,12899780,582732,1661724058,0,58.0
3,12899780,973453,1661724109,0,51.0
4,12899780,736515,1661724136,0,27.0
...,...,...,...,...,...
6928118,14571577,1141710,1662328774,0,0.0
6928119,14571578,519105,1662328775,0,0.0
6928120,14571579,739876,1662328775,0,0.0
6928121,14571580,202353,1662328781,0,0.0


In [9]:
from tqdm import tqdm

def add_subsession(df, drop = False):
  cur_session = -1
  cur_subsession = -1
  subsessions = []
  max_afk = 2 * 3600

  count = 0

  remove = []
  for row in tqdm(df.itertuples(), total = df.shape[0]):
    if row.session != cur_session:

      if cur_subsession != -1 and cur_subsession >= max_session:
        session_tracking = np.array(session_tracking)
        m = np.max(session_tracking)
        upto_remove = m - max_session
        num_redundant = np.sum(np.where(session_tracking <= upto_remove, 1, 0))
        remove += np.arange(start_idx, start_idx + num_redundant).tolist()
      
      if count > 39:
        subsession_tracking = np.array(subsession_tracking)
        m = np.max(subsession_tracking)
        upto_remove = m - 9
        num_redundant = np.sum(np.where(subsession_tracking <= upto_remove, 1, 0))
        remove += np.arange(sub_start_idx, sub_start_idx + num_redundant).tolist()

      #reset sesioon
      cur_session = row.session
      cur_subsession = 0
      
      start_idx = row[0]
      session_tracking = []

      #reset subsession
      sub_start_idx = row[0]
      count = 0 
      subsession_tracking = []





    if row.diff > max_afk:
      if count > 39:
        subsession_tracking = np.array(subsession_tracking)
        m = np.max(subsession_tracking)
        upto_remove = m - 9
        num_redundant = np.sum(np.where(subsession_tracking <= upto_remove, 1, 0))
        remove += np.arange(sub_start_idx, sub_start_idx + num_redundant).tolist()
      
      #reset subsession
      sub_start_idx = row[0]
      count = 0
      subsession_tracking = []
      cur_subsession += 1


    else:
      
      count += 1
    
    subsession_tracking.append(count)
    session_tracking.append(cur_subsession)
    subsessions.append(cur_subsession)



  df['subsession'] = subsessions
  if drop:
    print(f'Delete {len(remove)} rows')
    df = df.drop(index = remove, axis = 0)

  del subsessions
  return df

In [10]:
train_df = add_subsession(train_df, drop = True)
train_df

100%|██████████| 163555218/163555218 [03:30<00:00, 777344.63it/s]


Delete 19342873 rows


,session,aid,ts,type,diff,subsession
63,0,1629651,1659964140,0,0.0,0
64,0,1768568,1660015113,0,50973.0,1
65,0,1318324,1660015128,0,15.0,1
66,0,1840418,1660024261,0,9133.0,2
67,0,1813509,1660024355,0,94.0,2
...,...,...,...,...,...,...
216716091,12899776,1737908,1661723987,0,15.0,0
216716092,12899777,384045,1661723976,0,0.0,0
216716093,12899777,384045,1661723986,0,10.0,0
216716094,12899778,561560,1661723983,0,0.0,0


In [11]:
test_df = add_subsession(test_df, drop = False)
test_df

100%|██████████| 6928123/6928123 [00:09<00:00, 753037.23it/s]


,session,aid,ts,type,diff,subsession
0,12899779,59625,1661724000,0,0.0,0
1,12899780,1142000,1661724000,0,0.0,0
2,12899780,582732,1661724058,0,58.0,0
3,12899780,973453,1661724109,0,51.0,0
4,12899780,736515,1661724136,0,27.0,0
...,...,...,...,...,...,...
6928118,14571577,1141710,1662328774,0,0.0,0
6928119,14571578,519105,1662328775,0,0.0,0
6928120,14571579,739876,1662328775,0,0.0,0
6928121,14571580,202353,1662328781,0,0.0,0


# Save df

In [9]:
train_df.to_parquet('/home/anhphantq/splitted_unidata_sub/train_v1.parquet', index = False)


In [10]:
test_df.to_parquet('/home/anhphantq/splitted_unidata_sub/test_v1.parquet', index = False)

In [12]:
train_df.to_parquet('/home/anhphantq/unidata_sub/train_v1.parquet', index = False)

In [13]:
test_df.to_parquet('/home/anhphantq/unidata_sub/test_v1.parquet', index = False)

# align subsession to 0 -> 64

In [14]:
def substract_redudant_session(df, test = False):
  min_sub = df.loc[:, ['session','subsession']].groupby('session').min()
  df = df.merge(min_sub, on = 'session', how = 'left')
  df['subsession'] = df['subsession_x'] - df['subsession_y']
  if not test:
    assert df['subsession'].max() == max_session - 1
  assert df['subsession'].min() == 0
  del df['subsession_x'], df['subsession_y']
  return df


In [15]:
train_df = substract_redudant_session(train_df)
train_df

,session,aid,ts,type,diff,subsession
0,0,1629651,1659964140,0,0.0,0
1,0,1768568,1660015113,0,50973.0,1
2,0,1318324,1660015128,0,15.0,1
3,0,1840418,1660024261,0,9133.0,2
4,0,1813509,1660024355,0,94.0,2
...,...,...,...,...,...,...
144212340,12899776,1737908,1661723987,0,15.0,0
144212341,12899777,384045,1661723976,0,0.0,0
144212342,12899777,384045,1661723986,0,10.0,0
144212343,12899778,561560,1661723983,0,0.0,0


In [16]:
test_df = substract_redudant_session(test_df, test = True)
test_df

,session,aid,ts,type,diff,subsession
0,12899779,59625,1661724000,0,0.0,0
1,12899780,1142000,1661724000,0,0.0,0
2,12899780,582732,1661724058,0,58.0,0
3,12899780,973453,1661724109,0,51.0,0
4,12899780,736515,1661724136,0,27.0,0
...,...,...,...,...,...,...
6928118,14571577,1141710,1662328774,0,0.0,0
6928119,14571578,519105,1662328775,0,0.0,0
6928120,14571579,739876,1662328775,0,0.0,0
6928121,14571580,202353,1662328781,0,0.0,0


# save vprops

In [17]:
def vprops(df, path):
  vprop = df.loc[:, ['session','subsession', 'ts']].groupby('session').agg({
      'ts': ['min', 'max', 'count'],
      'subsession': 'max'
  })
  vprop['consistency'] = vprop.iloc[:, 1] - vprop.iloc[:, 0]
  vprop.reset_index().to_parquet(path, index = False)


In [16]:
vprops(train_df, '/home/anhphantq/splitted_unidata_sub/train_vprops_v1.parquet')

In [17]:
vprops(test_df, '/home/anhphantq/splitted_unidata_sub/test_vprops_v1.parquet')

In [18]:
vprops(train_df, '/home/anhphantq/unidata_sub/train_vprops_v1.parquet')

In [19]:
vprops(test_df, '/home/anhphantq/unidata_sub/test_vprops_v1.parquet')

# transform

In [20]:
import numpy as np
from tqdm import tqdm
TRAIN_TS_MIN = train_df.ts.min()
def transform_df(df):
  df['ts'] = df['ts'] - TRAIN_TS_MIN
  df['clicks'] = np.where(df['type'] == 0, 1, 0)
  df['carts'] = np.where(df['type'] == 1, 1, 0)
  df['orders'] = np.where(df['type'] == 2, 1, 0)
      
  assert df['clicks'].sum() + df['carts'].sum() + df['orders'].sum() == df.shape[0]
  del df['type'], df['diff']




In [21]:
transform_df(train_df)
train_df

,session,aid,ts,subsession,clicks,carts,orders
0,0,1629651,54540,0,1,0,0
1,0,1768568,105513,1,1,0,0
2,0,1318324,105528,1,1,0,0
3,0,1840418,114661,2,1,0,0
4,0,1813509,114755,2,1,0,0
...,...,...,...,...,...,...,...
144212340,12899776,1737908,1814387,0,1,0,0
144212341,12899777,384045,1814376,0,1,0,0
144212342,12899777,384045,1814386,0,1,0,0
144212343,12899778,561560,1814383,0,1,0,0


In [22]:
transform_df(test_df)
test_df

,session,aid,ts,subsession,clicks,carts,orders
0,12899779,59625,1814400,0,1,0,0
1,12899780,1142000,1814400,0,1,0,0
2,12899780,582732,1814458,0,1,0,0
3,12899780,973453,1814509,0,1,0,0
4,12899780,736515,1814536,0,1,0,0
...,...,...,...,...,...,...,...
6928118,14571577,1141710,2419174,0,1,0,0
6928119,14571578,519105,2419175,0,1,0,0
6928120,14571579,739876,2419175,0,1,0,0
6928121,14571580,202353,2419181,0,1,0,0


# Group to edges

In [23]:

def encode(x):
  return np.sum(2 ** np.unique(x))

def group_to_egdes(args):
  df, batchname = args
  return df.groupby(['session', 'aid']).agg({ 
      'ts': ['last','first'],
      'clicks': 'sum',
      'carts': 'sum',
      'orders': 'sum',
      'subsession': encode
  }).reset_index(), batchname
  

In [24]:
from tqdm import tqdm
import numpy as np
step = train_df.shape[0] // 64
start = [i * step for i in range(64)]
end = [i + step for i in start]
end[-1] = train_df.shape[0] 


In [25]:
batches = [train_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]
batch_names = [i for i in range(len(batches))]


In [ ]:
import multiprocessing
import os

with multiprocessing.Pool(64) as pool:
  for rs, batch in tqdm(pool.imap_unordered(group_to_egdes,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
    rs.to_parquet(f'/home/anhphantq/splitted_unidata_sub/edges_sub_frag_{batch}_v1.parquet', index = False)


100%|██████████| 64/64 [01:58<00:00,  1.86s/it] 


In [ ]:
import numpy as np
test_df.groupby(['session', 'aid']).agg({ 
    'ts': ['last','first'],
    'clicks': 'sum',
    'carts': 'sum',
    'orders': 'sum',
    'subsession': encode
}).reset_index().to_parquet('/home/anhphantq/splitted_unidata_sub/edges_test_sub_v1.parquet', index = False)

In [26]:
import multiprocessing
import os

with multiprocessing.Pool(64) as pool:
  for rs, batch in tqdm(pool.imap_unordered(group_to_egdes,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
    rs.to_parquet(f'/home/anhphantq/unidata_sub/edges_sub_frag_{batch}_v1.parquet', index = False)
  


100%|██████████| 64/64 [02:06<00:00,  1.98s/it] 


In [27]:
import numpy as np
test_df.groupby(['session', 'aid']).agg({ 
    'ts': ['last','first'],
    'clicks': 'sum',
    'carts': 'sum',
    'orders': 'sum',
    'subsession': encode
}).reset_index().to_parquet('/home/anhphantq/unidata_sub/edges_test_sub_v1.parquet', index = False)

RECENT FEATURES

In [26]:
import numpy as np
def recent_7days(df):
    unique_aids = np.unique(df.aid.values)
    df['date'] = pd.to_datetime(df.ts * 1e9)#7 recent days
    df['dayofyear'] = df.date.dt.dayofyear
    
    dayofyear_max = df['dayofyear'].max()
    dayofyear_min = dayofyear_max - 6
    
    df = df.loc[(df.dayofyear >= dayofyear_min) & (df.dayofyear <= dayofyear_max)]
    
    df['dayofyear_type'] = df.dayofyear * 10 + df.type
    item_agg = pd.DataFrame(df[['aid', 'dayofyear_type', 'ts']].groupby(['aid', 'dayofyear_type'])['ts'].count()).rename(columns = {'ts': 'count'})


    dates = pd.DataFrame({'dayofyear': np.arange(dayofyear_min, dayofyear_max + 1), 'tmp': 1})
    types = pd.DataFrame({'types': [0,1,2], 'tmp' : 1})
    aids = pd.DataFrame({'aid': np.unique(df.aid), 'tmp': 1})
    dates = dates.merge(types, on = 'tmp')

    dates = aids.merge(dates, on = 'tmp')
    del dates['tmp']
    dates['dayofyear_type'] = dates['dayofyear'] * 10 + dates['types']
    del dates['types']
    dates['count'] = np.nan
    dates = dates.groupby(['aid', 'dayofyear_type']).first()


    cols = []
    for i in range(7,0,-1):
        for j in range(3):
            cols.append(f'recent_day{i}_type{j}')
 
    
    item_agg = dates.combine_first(item_agg).fillna(0).reset_index()
    item_agg['count'] = item_agg['count'].values.astype(int)
    recent_7days = pd.DataFrame(data = item_agg[['aid', 'count']].groupby('aid')['count'].apply(list).tolist(), columns = [col for col in cols])
    
    
    item_agg = pd.concat((pd.DataFrame({
        'aid': np.unique(item_agg.aid)
    }), recent_7days), axis = 1)
    
    
    item_agg = pd.DataFrame({'aid': unique_aids, **dict(
        [(col, np.nan) for col in cols], 
    )}).set_index('aid').combine_first(item_agg.set_index('aid')).fillna(0)
    return item_agg.reset_index()

In [27]:
item_agg = recent_7days(train_df)

In [ ]:
item_agg

,aid,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,...,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2
0,0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,206.0,18.0,5.0,99.0,11.0,0.0,13.0,0.0,0.0,...,3.0,211.0,27.0,6.0,228.0,19.0,2.0,335.0,32.0,7.0
4,4,8.0,1.0,0.0,12.0,1.0,0.0,7.0,0.0,0.0,...,0.0,8.0,1.0,0.0,10.0,0.0,0.0,14.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839553,1855598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1839554,1855599,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1839555,1855600,2.0,0.0,0.0,4.0,0.0,0.0,3.0,2.0,0.0,...,0.0,4.0,1.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0
1839556,1855601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_agg.to_parquet("/home/anhphantq/splitted_unidata_sub/item_props_v1.parquet", index = False)

In [ ]:
item_agg.to_parquet("/home/anhphantq/unidata_sub/item_props_v1.parquet", index = False)

ITEM GLOBAL FEATURES

In [ ]:
TRAIN_TS_MIN = train_df.ts.min()

train_df

,session,aid,ts,type,date,dayofyear
63,0,1629651,1659964140,0,2022-08-08 13:09:00,220
64,0,1768568,1660015113,0,2022-08-09 03:18:33,221
65,0,1318324,1660015128,0,2022-08-09 03:18:48,221
66,0,1840418,1660024261,0,2022-08-09 05:51:01,221
67,0,1813509,1660024355,0,2022-08-09 05:52:35,221
...,...,...,...,...,...,...
216716091,12899776,1737908,1661723987,0,2022-08-28 21:59:47,240
216716092,12899777,384045,1661723976,0,2022-08-28 21:59:36,240
216716093,12899777,384045,1661723986,0,2022-08-28 21:59:46,240
216716094,12899778,561560,1661723983,0,2022-08-28 21:59:43,240


In [ ]:
def global_item_features(df):
    df['scaled_ts'] = (df.ts - TRAIN_TS_MIN) / 3600 / 24 
    max_ts = df.scaled_ts.max()
    df['time_decay']  =  0.9 ** (max_ts - df.scaled_ts )
    df['click'] = np.where(df['type'] == 0, df.time_decay, 0)  
    df['cart'] = np.where(df['type'] == 1, df.time_decay, 0)
    df['order'] = np.where(df['type'] == 2, df.time_decay, 0)
    return df[['aid','scaled_ts', 'time_decay', 'click', 'cart', 'order']].groupby('aid').agg({
    'scaled_ts': ['max', 'min'], 
    'time_decay': 'sum', 
    'click': 'sum',
    'cart' : 'sum',
    'order': 'sum',
}).reset_index()

In [ ]:
item_glob = global_item_features(train_df)

In [ ]:
item_glob

aid  scaled_ts             time_decay        click        cart  \
                        max       min          sum          sum         sum   
0              0  20.863345  1.452789    13.572334    13.572334    0.000000   
1              1  17.508866  2.738449     7.465299     7.319284    0.146015   
2              2  19.611991  4.524248     6.230244     6.230244    0.000000   
3              3  20.984317  0.434826  1461.256950  1322.898875  115.314236   
4              4  20.898519  0.388796    79.325614    75.313527    4.012088   
...          ...        ...       ...          ...          ...         ...   
1839553  1855598  17.944850  0.741273     2.034551     2.034551    0.000000   
1839554  1855599  19.679792  3.639051     4.168844     4.168844    0.000000   
1839555  1855600  20.814815  1.492014    27.600753    25.150745    2.321962   
1839556  1855601  12.285602  0.274120     7.111232     6.075555    1.035678   
1839557  1855602  20.750035  3.716285     7.147244     7.147244    0.000000   

             order  
               sum  
0         0.000000  
1         0.000000  
2         0.000000  
3        23.043839  
4         0.000000  
...            ...  
1839553   0.000000  
1839554   0.000000  
1839555   0.128045  
1839556   0.000000  
1839557   0.000000  

[1839558 rows x 7 columns]

In [ ]:
item_glob.to_parquet("/home/anhphantq/unidata_sub/item_glob_v1.parquet", index = False)

In [ ]:
item_glob.to_parquet("/home/anhphantq/splitted_unidata_sub/item_glob_v1.parquet", index = False)

RECENT INTERACT FEATURES

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import numpy as np
def recent_7days_inter(args):
    df, batchname = args
    df['date'] = pd.to_datetime(df.ts * 1e9)#7 recent days
    df['dayofyear'] = df.date.dt.dayofyear

    dayofyear_max = df['dayofyear'].max()
    dayofyear_min = dayofyear_max - 6

    df = df.loc[(df.dayofyear >= dayofyear_min) & (df.dayofyear <= dayofyear_max)]

    df['dayofyear_type'] = df.dayofyear * 10 + df.type
    item_agg = pd.DataFrame(df[['aid', 'dayofyear_type', 'ts']].groupby(['aid', 'dayofyear_type'])['ts'].count()).rename(columns = {'ts': 'count'})


    dates = pd.DataFrame({'dayofyear': np.arange(dayofyear_min, dayofyear_max + 1), 'tmp': 1})
    types = pd.DataFrame({'types': [0,1,2], 'tmp' : 1})
    dates = dates.merge(types, on = 'tmp')
    unique_user_aid = df[['session', 'aid']].groupby(['session', 'aid']).first().reset_index()
    unique_user_aid['tmp'] = 1
    dates = unique_user_aid.merge(dates, on = 'tmp')
    del dates['tmp']

    dates ['session_dayofyear_type'] = (dates.dayofyear * 10 + dates.types) * 100000000 + dates.session
    del dates['dayofyear'], dates['types'], dates['session']
    dates = dates.groupby(['aid', 'session_dayofyear_type']).first()
    dates['count'] = np.nan
    df['session_dayofyear_type'] =  df.dayofyear_type * 100000000 + df.session
    df = df[['session_dayofyear_type','aid', 'ts']].groupby(['session_dayofyear_type', 'aid']).count().rename(columns = {'ts': 'count'})

    cols = []
    for i in range(7,0,-1):
        for j in range(3):
            cols.append(f'recent_inter_day{i}_type{j}')
            
    item_agg = dates.combine_first(df).fillna(0).reset_index()
    item_agg['count'] = item_agg['count'].values.astype(int)
    item_agg['session'] = item_agg['session_dayofyear_type'] % 100000000
    
    recent_7days = pd.DataFrame(data = item_agg[['session', 'aid', 'count']].groupby(['session', 'aid'])['count'].apply(list).tolist(), columns = [col for col in cols])
    index = item_agg[['session', 'aid']].groupby(['session', 'aid']).first()
    recent_7days = pd.concat((index.reset_index(), recent_7days), axis = 1).set_index(['session', 'aid']).sort_index()
    return recent_7days, batchname

In [5]:
from tqdm import tqdm
import numpy as np

step = test_df.shape[0] // 64
start = [i * step for i in range(64)]
end = [i + step for i in start]
end[-1] = test_df.shape[0] 



In [6]:
batches = [test_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]
batch_names = [i for i in range(len(batches))]


In [7]:
del test_df

In [8]:
import multiprocessing
import os
assert os.path.exists('/home/anhphantq/splitted_unidata_sub/interact')
pool = multiprocessing.Pool(64)
for rs, batch in tqdm(pool.imap_unordered(recent_7days_inter,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
  rs.to_parquet(f'/home/anhphantq/splitted_unidata_sub/interact/recent_interacts_frag_{batch}_v1.parquet', index = True)
  
pool.close()
pool.join()

100%|██████████| 64/64 [00:23<00:00,  2.67it/s]


In [8]:
import multiprocessing
import os
assert os.path.exists('/home/anhphantq/unidata_sub/interact')
pool = multiprocessing.Pool(64)
for rs, batch in tqdm(pool.imap_unordered(recent_7days_inter,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
  rs.to_parquet(f'/home/anhphantq/unidata_sub/interact/recent_interacts_frag_{batch}_v1.parquet', index = True)
  
pool.close()
pool.join()

100%|██████████| 64/64 [00:21<00:00,  2.94it/s]


User features

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 150)

In [6]:
test_df

,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0
...,...,...,...,...
6928118,14571577,1141710,1662328774,0
6928119,14571578,519105,1662328775,0
6928120,14571579,739876,1662328775,0
6928121,14571580,202353,1662328781,0


In [7]:
test_df['clicks'] = np.where(test_df['type'] == 0, 1, 0)
test_df['carts'] = np.where(test_df['type'] == 1, 1, 0)
test_df['orders'] = np.where(test_df['type'] == 2, 1, 0)

In [8]:

test_df['user_clicks_cnt'] = test_df.groupby(['session'])['clicks'].transform('sum')
test_df['user_carts_cnt'] = test_df.groupby(['session'])['carts'].transform('sum')
test_df['user_orders_cnt'] = test_df.groupby(['session'])['orders'].transform('sum')
    


In [9]:
test_df['user_total_actions_cnt'] = test_df['user_clicks_cnt'] + test_df['user_carts_cnt'] + test_df['user_orders_cnt']

In [10]:
from tqdm.notebook import tqdm
for i in ['clicks', 'carts', 'orders']:
    test_df['user_' + i + '_ratio'] = test_df['user_' + i + '_cnt'] / test_df['user_total_actions_cnt']

In [11]:
test_df['user_ss_ts_max'] = test_df.groupby('session')['ts'].transform('max')
test_df['user_ss_ts_min'] = test_df.groupby('session')['ts'].transform('min')
test_df['user_ss_ts_mean'] = test_df.groupby('session')['ts'].transform('mean')
test_df['user_ss_ts_mean'] = test_df['user_ss_ts_mean'].astype('int32')

In [12]:
test_df = test_df.iloc[:, [0] + [i for i in range(7, test_df.shape[1])]]

In [13]:
test_df = test_df.groupby('session').first()

In [14]:
test_df

,user_clicks_cnt,user_carts_cnt,user_orders_cnt,user_total_actions_cnt,user_clicks_ratio,user_carts_ratio,user_orders_ratio,user_ss_ts_max,user_ss_ts_min,user_ss_ts_mean
session,,,,,,,,,,
12899779,1,0,0,1,1.000000,0.000000,0.000000,1661724000,1661724000,1661724000
12899780,5,0,0,5,1.000000,0.000000,0.000000,1661724155,1661724000,1661724091
12899781,10,1,0,11,0.909091,0.090909,0.000000,1662060160,1661724000,1661838558
12899782,46,16,8,70,0.657143,0.228571,0.114286,1661803953,1661724000,1661783634
12899783,11,0,0,11,1.000000,0.000000,0.000000,1662041140,1661724000,1661770355
...,...,...,...,...,...,...,...,...,...,...
14571577,1,0,0,1,1.000000,0.000000,0.000000,1662328774,1662328774,1662328774
14571578,1,0,0,1,1.000000,0.000000,0.000000,1662328775,1662328775,1662328775
14571579,1,0,0,1,1.000000,0.000000,0.000000,1662328775,1662328775,1662328775


In [39]:
test_df.to_parquet('/home/anhphantq/splitted_unidata_sub/user_features.parquet', index = True)

In [15]:
test_df.to_parquet('/home/anhphantq/unidata_sub/user_features.parquet', index = True)

Item features

In [16]:
train_df

,session,aid,ts,type
63,0,1629651,1659964140,0
64,0,1768568,1660015113,0
65,0,1318324,1660015128,0
66,0,1840418,1660024261,0
67,0,1813509,1660024355,0
...,...,...,...,...
216716091,12899776,1737908,1661723987,0
216716092,12899777,384045,1661723976,0
216716093,12899777,384045,1661723986,0
216716094,12899778,561560,1661723983,0


In [17]:
import numpy as np
train_df['clicks'] = np.where(train_df['type'] == 0, 1, 0)
train_df['carts'] = np.where(train_df['type'] == 1, 1, 0)
train_df['orders'] = np.where(train_df['type'] == 2, 1, 0)

In [18]:
train_df['item_cnt'] = train_df.groupby('aid')['aid'].transform('count')


In [19]:

train_df['item_clicks_cnt'] = train_df.groupby(['aid'])['clicks'].transform('sum')
train_df['item_carts_cnt'] = train_df.groupby(['aid'])['carts'].transform('sum')
train_df['item_orders_cnt'] = train_df.groupby(['aid'])['orders'].transform('sum')
    


In [20]:
train_df

,session,aid,ts,type,clicks,carts,orders,item_cnt,item_clicks_cnt,item_carts_cnt,item_orders_cnt
63,0,1629651,1659964140,0,1,0,0,1550,1475,66,9
64,0,1768568,1660015113,0,1,0,0,1524,1436,88,0
65,0,1318324,1660015128,0,1,0,0,331,280,45,6
66,0,1840418,1660024261,0,1,0,0,10881,10146,634,101
67,0,1813509,1660024355,0,1,0,0,4233,4028,175,30
...,...,...,...,...,...,...,...,...,...,...,...
216716091,12899776,1737908,1661723987,0,1,0,0,307,274,21,12
216716092,12899777,384045,1661723976,0,1,0,0,2207,2094,87,26
216716093,12899777,384045,1661723986,0,1,0,0,2207,2094,87,26
216716094,12899778,561560,1661723983,0,1,0,0,271,235,28,8


In [21]:
train_df['item_total_actions_cnt'] = train_df['item_clicks_cnt'] + train_df['item_carts_cnt'] + train_df['item_orders_cnt']

In [22]:
from tqdm.notebook import tqdm
for i in ['clicks', 'carts', 'orders']:
    train_df['item_' + i + '_ratio'] = train_df['item_' + i + '_cnt'] / train_df['item_total_actions_cnt']

In [23]:
train_df

,session,aid,ts,type,clicks,carts,orders,item_cnt,item_clicks_cnt,item_carts_cnt,item_orders_cnt,item_total_actions_cnt,item_clicks_ratio,item_carts_ratio,item_orders_ratio
63,0,1629651,1659964140,0,1,0,0,1550,1475,66,9,1550,0.951613,0.042581,0.005806
64,0,1768568,1660015113,0,1,0,0,1524,1436,88,0,1524,0.942257,0.057743,0.000000
65,0,1318324,1660015128,0,1,0,0,331,280,45,6,331,0.845921,0.135952,0.018127
66,0,1840418,1660024261,0,1,0,0,10881,10146,634,101,10881,0.932451,0.058267,0.009282
67,0,1813509,1660024355,0,1,0,0,4233,4028,175,30,4233,0.951571,0.041342,0.007087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216716091,12899776,1737908,1661723987,0,1,0,0,307,274,21,12,307,0.892508,0.068404,0.039088
216716092,12899777,384045,1661723976,0,1,0,0,2207,2094,87,26,2207,0.948799,0.039420,0.011781
216716093,12899777,384045,1661723986,0,1,0,0,2207,2094,87,26,2207,0.948799,0.039420,0.011781
216716094,12899778,561560,1661723983,0,1,0,0,271,235,28,8,271,0.867159,0.103321,0.029520


In [24]:
train_df = train_df.iloc[:, [1] + [i for i in range(7, train_df.shape[1])]]

In [25]:
train_df = train_df.groupby('aid').first()

In [26]:
train_df

,item_cnt,item_clicks_cnt,item_carts_cnt,item_orders_cnt,item_total_actions_cnt,item_clicks_ratio,item_carts_ratio,item_orders_ratio
aid,,,,,,,,
0,33,33,0,0,33,1.000000,0.000000,0.000000
1,31,30,1,0,31,0.967742,0.032258,0.000000
2,16,16,0,0,16,1.000000,0.000000,0.000000
3,2540,2316,186,38,2540,0.911811,0.073228,0.014961
4,163,155,8,0,163,0.950920,0.049080,0.000000
...,...,...,...,...,...,...,...,...
1855598,6,6,0,0,6,1.000000,0.000000,0.000000
1855599,10,10,0,0,10,1.000000,0.000000,0.000000
1855600,69,64,4,1,69,0.927536,0.057971,0.014493


In [44]:
train_df.to_parquet('/home/anhphantq/splitted_unidata_sub/item_features.parquet', index = True)

In [27]:
train_df.to_parquet('/home/anhphantq/unidata_sub/item_features.parquet', index = True)